In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def get_graph(net, remove_autoconnections=True):
    df = pd.read_csv(base+"edges.csv", index_col=0)
    df.reset_index(inplace=True)
    if remove_autoconnections:
        df = df[df['source'] != df['target']] 
    df_cls =  pd.read_csv(f"{base+net}classes.csv", index_col=0)
    
    g = ig.Graph.DataFrame(df, directed=False)
    
    
    colors = []
    for patient in g.vs["name"]:

        cls = df_cls.loc[patient, :]['class']

        if cls == "stage1":
            colors.append("green")
        elif cls ==  "stage2":
            colors.append("blue")
        elif cls ==  "stage3":
            colors.append("magenta")
        elif cls ==  "stage4":
            colors.append("red")
        elif cls ==  "control":
            colors.append("yellow")

    g.vs["color"] = colors
    return g




def plot_graph(net, remove_autoconnections=True):
    
    df = pd.read_csv(base+net+"edges.csv", index_col=0)
    df.reset_index(inplace=True)
    #if remove_autoconnections:
    #    df = df[df['source'] != df['target']] 
    df_cls =  pd.read_csv(f"{base+net}classes.csv", index_col=0)
    
    g = ig.Graph.DataFrame(df, directed=False)
    
    
    colors = []
    for patient in g.vs["name"]:

        cls = df_cls.loc[patient, :]['class']

        if cls == "stage1":
            colors.append("green")
        elif cls ==  "stage2":
            colors.append("blue")
        elif cls ==  "stage3":
            colors.append("magenta")
        elif cls == "stage23":
            colors.append("blue")
        elif cls ==  "stage4":
            colors.append("red")
        elif cls ==  "control":
            colors.append("yellow")

    g.vs["color"] = colors
    
    
    if remove_autoconnections:
        for i in range(len(g.vs)):
            g.delete_edges([(i,i)])
    
    """
    edge_colors = []
    for e in g.es:
        if e['weight'] >= 0:
            edge_colors.append("black")
        else:
            edge_colors.append("orange")
    g.es["color"] = edge_colors
    """
    for e in g.es:
        n1, n2 = e.tuple
        if g.vs[n1]['color'] == g.vs[n2]['color']:
            e['color'] = 'black'
        else:
            e['color'] = 'orange'
    
    
    layout = g.layout(layout='auto')
    out = ig.plot(g, layout = layout, bbox = (700, 700), margin = 10, vertex_size=10)
    if remove_autoconnections:
        out.save(base+net+"graph_plot.png")
    else:
        out.save(base+net+"graph_plot_autoconnections.png")
    return out, g


def get_graph_cis(g):
    
    ci_total = len(g.es)
    ci_right = 0
    
    colors = {
        "yellow": 0,
        "yellow_match": 0,
        "green": 0,
        "green_match": 0,
        "blue": 0,
        "blue_match": 0,
        "magenta": 0,
        "magenta_match": 0,
        "red": 0,
        "red_match": 0
    }
    
    for e in g.es:
        n1, n2 = e.tuple
        n1 = g.vs[n1]
        n2 = g.vs[n2]

        colors[n1["color"]] +=1
        colors[n2["color"]] +=1
        if colors[n1["color"]] == colors[n2["color"]]:
            colors[n2["color"]] -= 1
            colors[n2["color"]+"_match"] += 1
            ci_right +=1
    
    return (ci_right/ci_total, colors["yellow_match"]/colors["yellow"], 
            colors["green_match"]/colors["green"], colors["blue_match"]/colors["blue"],
            colors["magenta_match"]/colors["magenta"], colors["red_match"]/colors["red"])


def get_graph_ces(cis):
    ces = [(-1*ci)+1 for ci in cis]
    return ces

def get_ci_ce_dfs():
    ci_dict = {}
    ce_dict = {}
    for n in nets:
        g = get_graph(n)
        cis = get_graph_cis(g)
        ces = get_graph_ces(cis)
        ci_dict[n] = [ci for ci in cis]
        ce_dict[n] = [ce for ce in ces]

    cols = ["Total", "Stage 1", "Stage 2_3", "Stage 4"]

    ci_df = pd.DataFrame.from_dict(ci_dict).T
    ci_df.columns = cols
    ce_df = pd.DataFrame.from_dict(ce_dict).T
    ce_df.columns = cols
    
    return ci_df, ce_df


def plot_cis():
    sns.set_theme(style="whitegrid")
    ci_df, _ = get_ci_ce_dfs()
    dfs = []
    
    for i, net in enumerate(nets):
        cis = ci_df.iloc[[i], :].values[0]
        cls = ci_df.columns.values
        df = pd.DataFrame()
        df['CI'] = cis
        df['Class'] = cls
        df['Network'] = [net]*len(v)
        dfs.append(df)
        
        cis_plt = sns.barplot(x="Class", y="CI", data=df).set_title(net)
        plt.ylim(0, 1)
        plt.savefig(base+net+"cis.png")
        plt.clf()
    
    df = pd.concat(dfs)
    df_not_total = df.loc[df["Class"] != "Total"]
    sns.barplot(x="Class", y="CI", data=df_not_total, hue="Network").set_title("CI Class Comparison")
    plt.ylim(0, 1)
    plt.savefig(base+"ci_class_comparison.png")
    plt.clf()
    
    
    df_only_total = df.loc[df["Class"] == "Total"]
    sns.barplot(x="Network", y="CI", data=df_only_total).set_title("CI Comparison")
    plt.ylim(0, 1)
    plt.savefig(base+"ci_comparison.png")
    plt.clf()
    
    return 

In [12]:

base = "C:/Users/colombelli/Desktop/TCC/data/TCGA/KIRC/"
nets = [
    "stage_1_2_3_4/07_005_100feat/",
    "stage_1_2_3_4/06_005_100feat/",
    "stage_1_2_3_4/05_005_100feat/",
    "stage_1_2_3_4/04_005_100feat/",
    "stage_1_2_3_4/03_005_100feat/",
    "stage_1_2_3_4/02_005_100feat/",
    "stage_1_2_3_4/01_005_100feat/",
    "stage_1_2_3_4/0_005_100feat/"
    ]

for n in nets:
    out = plot_graph(n)
    
for n in nets:
    out = plot_graph(n, False)

In [6]:
cancers = ["COAD", "KIRC", "LUAD"]
r_thresholds = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8] 

for cancer in cancers:
    print(cancer)
    base = f"C:/Users/colombelli/Desktop/TCC/experiments/{cancer}/thresholds/"
    for r in r_thresholds:
        net = f"{str(r).replace('.', '')}_005/"
        out = plot_graph(net)

COAD
KIRC
LUAD
